In [18]:
# 데이터 가져오기

import json
from urllib.request import urlopen
import pandas as pd
import numpy as np
from datetime import date
date=str(date.today().year)+str(date.today().month)+str(date.today().day)
start_date=str(int(date)-100)
key='U8U9S9MEHH8NU1X1BRBV'
nara=['0000001','0000002','0000003','0000053']   # 나라 코드(0000001:us, 0000002:jp, 0000003:eu, 0000053:cn)
url=["http://ecos.bok.or.kr/api/StatisticSearch/"+key+"/json/kr/1/100000/036Y001/DD/20180401/date/0000001/?/?/",  # 환율
    "http://ecos.bok.or.kr/api/StatisticSearch/"+key+"/json/kr/1/100000/098Y001/MM/20190401/date/0101000/?/?/",   # 기준금리
    "http://ecos.bok.or.kr/api/StatisticSearch/"+key+"/json/kr/1/100000/022Y013/MM/20190401/date/000000/?/?/",    # 경상수지
    ]    

s_data=pd.DataFrame()
time=[]
us=[]
jp=[]
eu=[]
cn=[]

for i in range(len(nara)):
    data=json.loads(urlopen(url[0][:-12]+nara[i]+url[0][-5:]).read())
    data2=data['StatisticSearch']['row']
    for k in range(len(data2)):        
        if nara[i]==nara[0]:
            us.append(float(data2[k]['DATA_VALUE']))
            time.append(int(data2[k]['TIME']))
        elif nara[i]==nara[1]:
            jp.append(float(data2[k]['DATA_VALUE']))
        elif nara[i]==nara[2]:
            eu.append(float(data2[k]['DATA_VALUE']))
        else :
            cn.append(float(data2[k]['DATA_VALUE']))

s_data['time']=time
s_data['us']=us
s_data['jp']=jp
s_data['eu']=eu
s_data['cn']=cn
s_data['time']=pd.to_datetime(s_data.time.apply(lambda x: str(x)))
print(s_data)


          time      us       jp       eu      cn
0   2018-04-02  1064.4  1000.80  1311.07  169.80
1   2018-04-03  1058.1   999.67  1301.73  168.83
2   2018-04-04  1057.8   992.82  1298.08  168.60
3   2018-04-05  1057.6   990.68  1299.31  168.26
4   2018-04-06  1059.5   989.03  1297.41  168.56
..         ...     ...      ...      ...     ...
572 2020-07-27  1201.3  1133.03  1399.15  171.15
573 2020-07-28  1196.6  1135.89  1406.66  170.78
574 2020-07-29  1195.1  1137.05  1400.72  170.75
575 2020-07-30  1195.2  1138.39  1408.78  170.65
576 2020-07-31  1191.4  1137.32  1411.21  170.26

[577 rows x 5 columns]


In [19]:
# LSTM 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [20]:
# LSTM_US
import random as rn
import os
os.environ['PYTHONHASHSEED']=str(0)
np.random.seed(0)
tf.random.set_seed(0)
rn.seed(0)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(s_data[['us']])
scaled_train_data = scaler.transform(s_data[['us']])

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

n_input = 1
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=1)

lstm_model = Sequential()
lstm_model.add(LSTM(200, kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.01),activation='relu', input_shape=(n_input, n_features)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.fit_generator(generator,epochs=20)

lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(365):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
print(lstm_predictions)
usd=pd.DataFrame(lstm_predictions)

Epoch 1/20
576/576 [==============================] - 64s 111ms/step - loss: 12.5802
Epoch 2/20
576/576 [==============================] - 64s 110ms/step - loss: 6.5332
Epoch 3/20
576/576 [==============================] - 63s 109ms/step - loss: 3.2662
Epoch 4/20
576/576 [==============================] - 63s 109ms/step - loss: 1.5324
Epoch 5/20
576/576 [==============================] - 70s 122ms/step - loss: 0.6607
Epoch 6/20
576/576 [==============================] - 63s 109ms/step - loss: 0.2560
Epoch 7/20
576/576 [==============================] - 65s 113ms/step - loss: 0.0897
Epoch 8/20
576/576 [==============================] - 68s 117ms/step - loss: 0.0318
Epoch 9/20
576/576 [==============================] - 66s 114ms/step - loss: 0.0154
Epoch 10/20
576/576 [==============================] - 68s 118ms/step - loss: 0.0109
Epoch 11/20
576/576 [==============================] - 66s 115ms/step - loss: 0.0097
Epoch 12/20
576/576 [==============================] - 65s 113ms/step - l

In [21]:
lstm_model.save("usd_model.h5")

In [22]:
# LSTM_JP

os.environ['PYTHONHASHSEED']=str(0)
np.random.seed(0)
tf.random.set_seed(0)
rn.seed(0)

scaler = MinMaxScaler()
scaler.fit(s_data[['jp']])
scaled_train_data = scaler.transform(s_data[['jp']])


from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

n_input = 1
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=1)

lstm_model = Sequential()
lstm_model.add(LSTM(200, kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.01),activation='relu', input_shape=(n_input, n_features)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.fit_generator(generator,epochs=20)

lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(365):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
print(lstm_predictions)
jpy=pd.DataFrame(lstm_predictions)

Epoch 1/20
576/576 [==============================] - 72s 125ms/step - loss: 12.5962
Epoch 2/20
576/576 [==============================] - 65s 113ms/step - loss: 6.5502
Epoch 3/20
576/576 [==============================] - 64s 111ms/step - loss: 3.2780
Epoch 4/20
576/576 [==============================] - 63s 109ms/step - loss: 1.5402
Epoch 5/20
576/576 [==============================] - 64s 112ms/step - loss: 0.6660
Epoch 6/20
576/576 [==============================] - 65s 112ms/step - loss: 0.2598
Epoch 7/20
576/576 [==============================] - 1072s 2s/step - loss: 0.0921
Epoch 8/20
576/576 [==============================] - 67s 116ms/step - loss: 0.0334
Epoch 9/20
576/576 [==============================] - 67s 116ms/step - loss: 0.0177
Epoch 10/20
576/576 [==============================] - 72s 126ms/step - loss: 0.0131
Epoch 11/20
576/576 [==============================] - 68s 118ms/step - loss: 0.0115
Epoch 12/20
576/576 [==============================] - 66s 114ms/step - lo

In [23]:
lstm_model.save("jpy_model.h5")

In [24]:
# LSTM_CN

rn.seed(0)
os.environ['PYTHONHASHSEED']=str(0)
np.random.seed(0)
tf.random.set_seed(0)

scaler = MinMaxScaler()
scaler.fit(s_data[['cn']])
scaled_train_data = scaler.transform(s_data[['cn']])


from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

n_input = 1
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=1)

lstm_model = Sequential()
lstm_model.add(LSTM(200,  kernel_initializer='he_normal',kernel_regularizer=tf.keras.regularizers.l2(0.01),activation='relu', input_shape=(n_input, n_features)))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.fit_generator(generator,epochs=20)

lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(365):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
print(lstm_predictions)
cny=pd.DataFrame(lstm_predictions)

Epoch 1/20
576/576 [==============================] - 68s 117ms/step - loss: 12.5587
Epoch 2/20
576/576 [==============================] - 67s 116ms/step - loss: 6.5168
Epoch 3/20
576/576 [==============================] - 71s 123ms/step - loss: 3.2557
Epoch 4/20
576/576 [==============================] - 73s 127ms/step - loss: 1.5259
Epoch 5/20
576/576 [==============================] - 67s 117ms/step - loss: 0.6559
Epoch 6/20
576/576 [==============================] - 67s 116ms/step - loss: 0.2524
Epoch 7/20
576/576 [==============================] - 68s 118ms/step - loss: 0.0877
Epoch 8/20
576/576 [==============================] - 68s 118ms/step - loss: 0.0297
Epoch 9/20
576/576 [==============================] - 67s 117ms/step - loss: 0.0132
Epoch 10/20
576/576 [==============================] - 68s 118ms/step - loss: 0.0097
Epoch 11/20
576/576 [==============================] - 69s 120ms/step - loss: 0.0081
Epoch 12/20
576/576 [==============================] - 68s 118ms/step - l

In [25]:
lstm_model.save("cny_model.h5")

In [26]:
for i in range(2):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)
print(lstm_predictions)
cny=pd.DataFrame(lstm_predictions)

[[170.15419174]
 [170.05528378]
 [169.96282727]
 [169.87640146]
 [169.79561335]
 [169.72009545]
 [169.64950453]
 [169.5835194 ]
 [169.5218391 ]
 [169.46418349]
 [169.41028984]
 [169.35991274]
 [169.312823  ]
 [169.2688059 ]
 [169.22766177]
 [169.18920252]
 [169.15325335]
 [169.11965048]
 [169.08824054]
 [169.05888057]
 [169.03110872]
 [169.00318118]
 [168.97509737]
 [168.94685673]
 [168.91845808]
 [168.88990027]
 [168.8611833 ]
 [168.83230544]
 [168.8032661 ]
 [168.77406471]
 [168.7447001 ]
 [168.71517171]
 [168.68547837]
 [168.65561951]
 [168.62559396]
 [168.59540058]
 [168.56503877]
 [168.53450796]
 [168.503807  ]
 [168.47293473]
 [168.44189056]
 [168.41067334]
 [168.37928192]
 [168.34771571]
 [168.31597414]
 [168.28405604]
 [168.25196027]
 [168.21968565]
 [168.18723162]
 [168.15459702]
 [168.12178069]
 [168.08878205]
 [168.05560051]
 [168.02223436]
 [167.988683  ]
 [167.95494528]
 [167.92102004]
 [167.8869067 ]
 [167.85260411]
 [167.81811168]
 [167.78342769]
 [167.74855155]
 [167.71

In [27]:
print(usd)
print(jpy)
print(cny)

               0
0    1186.019214
1    1181.114808
2    1176.644628
3    1172.570226
4    1168.856578
..           ...
360  1120.046539
361  1120.046539
362  1120.046539
363  1120.046539
364  1120.046539

[365 rows x 1 columns]
               0
0    1131.117013
1    1125.638555
2    1120.791846
3    1116.274057
4    1111.989169
..           ...
360  1010.767860
361  1010.767860
362  1010.767860
363  1010.767860
364  1010.767860

[365 rows x 1 columns]
              0
0    170.154192
1    170.055284
2    169.962827
3    169.876401
4    169.795613
..          ...
362  166.176989
363  166.176989
364  166.176989
365  166.176989
366  166.176989

[367 rows x 1 columns]


In [28]:
def to_json(nation,jugi):
    pre_usd=dict()
    pre_jpy=dict()
    pre_cny=dict()
    if jugi=='일주일':
        if nation=='usd':
            for i in range(0,6):
                pre_usd['H{}'.format(i+1)]=usd[0][i]
                json_usd = json.dumps(pre_usd,separators=(';',':'))
                print(json_usd)
        elif nation=='jpy':
            for i in range(0,6):
                pre_jpy['H{}'.format(i+1)]=jpy[0][i]
                json_jpy = json.dumps(pre_jpy,separators=(';',':'))
                print(json_jpy)
        elif nation=='cny':
            for i in range(0,6):
                pre_cny['H{}'.format(i+1)]=cny[0][i]
                json_cny = json.dumps(pre_cny,separators=(';',':'))
                print(json_cny)
    if jugi=='한달':
        if nation=='usd':
            for i in range(0,31):
                pre_usd['H{}'.format(i+1)]=usd[0][i]
                json_usd = json.dumps(pre_usd,separators=(';',':'))
                print(json_usd)
        elif nation=='jpy':
            for i in range(0,31):
                pre_jpy['H{}'.format(i+1)]=jpy[0][i]
                json_jpy = json.dumps(pre_jpy,separators=(';',':'))
                print(json_jpy)
        elif nation=='cny':
            for i in range(0,31):
                pre_cny['H{}'.format(i+1)]=cny[0][i]
                json_cny = json.dumps(pre_cny,separators=(';',':'))
                print(json_cny)
    if jugi=='일년':
        if nation=='usd':
            for i in range(0,365):
                pre_usd['H{}'.format(i+1)]=usd[0][i]
                json_usd = json.dumps(pre_usd,separators=(';',':'))
                print(json_usd)
        elif nation=='jpy':
            for i in range(0,365):
                pre_jpy['H{}'.format(i+1)]=jpy[0][i]
                json_jpy = json.dumps(pre_jpy,separators=(';',':'))
                print(json_jpy)
        elif nation=='cny':
            for i in range(0,365):
                pre_cny['H{}'.format(i+1)]=cny[0][i]
                json_cny = json.dumps(pre_cny,separators=(';',':'))
                print(json_cny)
            

In [29]:
to_json('jpy','일주일')

{"H1":1131.1170128858091}
{"H1":1131.1170128858091;"H2":1125.6385551941396}
{"H1":1131.1170128858091;"H2":1125.6385551941396;"H3":1120.7918459963798}
{"H1":1131.1170128858091;"H2":1125.6385551941396;"H3":1120.7918459963798;"H4":1116.274056940079}
{"H1":1131.1170128858091;"H2":1125.6385551941396;"H3":1120.7918459963798;"H4":1116.274056940079;"H5":1111.989169279337}
{"H1":1131.1170128858091;"H2":1125.6385551941396;"H3":1120.7918459963798;"H4":1116.274056940079;"H5":1111.989169279337;"H6":1107.9195960175991}


In [30]:
to_json('usd','일주일')

{"H1":1186.0192141890525}
{"H1":1186.0192141890525;"H2":1181.1148081183433}
{"H1":1186.0192141890525;"H2":1181.1148081183433;"H3":1176.644627547264}
{"H1":1186.0192141890525;"H2":1181.1148081183433;"H3":1176.644627547264;"H4":1172.5702258408069}
{"H1":1186.0192141890525;"H2":1181.1148081183433;"H3":1176.644627547264;"H4":1172.5702258408069;"H5":1168.8565779685973}
{"H1":1186.0192141890525;"H2":1181.1148081183433;"H3":1176.644627547264;"H4":1172.5702258408069;"H5":1168.8565779685973;"H6":1165.4717356920241}
